## LLM Chain ##

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools that can be used to streamline the testing process. These tools can assist in running tests, generating reports, and identifying bugs or issues in the code. Langsmith can also provide support for running various types of tests, such as unit tests, integration tests, and end-to-end tests, to ensure that the software is functioning correctly. Additionally, Langsmith can help with creating test cases, managing test data, and improving test coverage to ensure thorough testing of the software.', response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 15, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'stop', 'logprobs': None})

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | llm
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in a variety of ways, as it is a versatile language technology tool. Here are some ways in which Langsmith can assist with testing:\n\n1. Automated Testing: Langsmith can be used to automate testing processes by generating test cases, executing tests, and verifying expected outcomes. This can help in speeding up the testing process and ensuring comprehensive test coverage.\n\n2. Test Data Generation: Langsmith can be used to generate test data for different scenarios, ensuring that the software being tested is exposed to a wide range of inputs and conditions.\n\n3. Test Scenario Generation: Langsmith can be used to generate test scenarios that cover various user interactions and use cases. This can help in ensuring that the software behaves as expected in different situations.\n\n4. Test Scripting: Langsmith can be used to write test scripts for automated testing frameworks, making it easier to create and maintain test scripts for regr

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool for testing in a few different ways:\n\n1. **Automated Testing**: Langsmith can be used to automate the testing process by generating test cases, executing them, and comparing the actual results with the expected results. This can help in identifying bugs and issues in the software more efficiently.\n\n2. **Test Data Generation**: Langsmith can be used to generate test data for various scenarios, allowing testers to cover a wide range of input values and edge cases in their testing. This can help in ensuring thorough test coverage and identifying potential issues early on.\n\n3. **Performance Testing**: Langsmith can be used to simulate different load conditions and test the performance of the software under various scenarios. This can help in identifying performance bottlenecks and optimizing the software for better performance.\n\n4. **Security Testing**: Langsmith can be used to generate test cases for security testing, such as testing for vulnerabi

## Retrieval Chain

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
embeddings = OpenAIEmbeddings()
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

# document_chain.invoke({
#     "input": "how can langsmith help with testing?",
#     "context": [Document(page_content="langsmith can let you visualize test results")]
# })

LangSmith provides features such as prototyping, debugging, initial test set creation, comparison view, playground environment for rapid iteration and experimentation, beta testing support, capturing feedback, annotating traces, adding runs to datasets, monitoring, and A/B testing. These features enable developers to test their LLM applications thoroughly, track performance metrics, and identify and address issues effectively throughout the development lifecycle.


## Conversation Retrieval Chain

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
embeddings = OpenAIEmbeddings()
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

retriever = vector.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM appli

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
embeddings = OpenAIEmbeddings()
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

retriever = vector.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly u